
# Laboratorio 3: Resolución de problemas de Programación Entera con Python

En este laboratorio, exploraremos cómo resolver problemas de **programación entera** (PE) utilizando Python, la biblioteca **Pyomo** y el solver **GLPK**, herramientas que ya hemos introducido en los laboratorios anteriores. 

La programación entera es una extensión de la programación lineal donde algunas o todas las variables de decisión deben tomar valores enteros o binarios. Esto es fundamental en situaciones donde las decisiones implican cantidades discretas, como seleccionar o no proyectos, asignar recursos indivisibles, o determinar rutas.

En este laboratorio:
1. Revisaremos cómo definir variables de decisión específicas para programación entera en Pyomo.
2. Veremos cómo adaptar los modelos existentes a esta nueva característica.

Aunque GLPK es capaz de resolver problemas de programación entera, ten en cuenta que los tiempos de resolución pueden aumentar significativamente con el tamaño del problema debido a la naturaleza combinatoria de estos modelos.



## Definición de variables de decisión en PE

En programación lineal, definíamos variables de decisión continuas usando la opción `NonNegativeReals` en Pyomo. En programación entera, necesitamos especificar de manera explícita si una variable es entera o binaria. Esto se hace utilizando:

- **`NonNegativeIntegers`**: Para variables que deben ser enteras no negativas.
- **`Binary`**: Para variables binarias (toman valores 0 o 1).

### Ejemplo

Supongamos que queremos definir:
1. Una variable entera $x_1$ que represente el número de unidades producidas de un producto.
2. Una variable binaria $x_2$ que indique si se selecciona o no un proyecto.

En Pyomo, estas variables se definirían de la siguiente manera:

```python
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir una variable entera no negativa
model.x1 = Var(within=NonNegativeIntegers)

# Definir una variable binaria
model.x2 = Var(within=Binary)
```

Estas definiciones permiten al solver manejar correctamente las restricciones de integridad durante la optimización.



### Ejemplo práctico

A continuación, resolveremos un problema clásico de programación entera: el problema de **selección de proyectos**.

Supongamos que una empresa desea seleccionar un subconjunto de proyectos para maximizar el beneficio total. Cada proyecto tiene un costo asociado y la empresa tiene un presupuesto limitado. Los datos del problema son:

| Proyecto | Beneficio (\$) | Costo (\$) |
|----------|----------------|------------|
| 1        | 50             | 20         |
| 2        | 60             | 30         |
| 3        | 70             | 40         |
| 4        | 80             | 50         |

La empresa tiene un presupuesto total de 100 \$ y desea maximizar el beneficio respetando este presupuesto. La decisión es binaria: incluir o no cada proyecto.


In [1]:

from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir los conjuntos de datos
proyectos = [1, 2, 3, 4]
beneficio = {1: 50, 2: 60, 3: 70, 4: 80}
costo = {1: 20, 2: 30, 3: 40, 4: 50}
presupuesto = 100

# Definir variables de decisión binarias
model.x = Var(proyectos, within=Binary)

# Definir la función objetivo
model.obj = Objective(expr=sum(beneficio[i] * model.x[i] for i in proyectos), sense=maximize)

# Definir la restricción del presupuesto
model.budget = Constraint(expr=sum(costo[i] * model.x[i] for i in proyectos) <= presupuesto)

# Resolver el modelo usando GLPK
solver = SolverFactory('glpk')
solver.solve(model)

# Imprimir los resultados
print("Proyectos seleccionados:")
for i in proyectos:
    if model.x[i]() > 0.5:  # La variable es binaria (0 o 1), usamos un umbral para considerar seleccionada
        print(f"Proyecto {i}")
print(f"Beneficio total: {model.obj()}")


Proyectos seleccionados:
Proyecto 1
Proyecto 2
Proyecto 4
Beneficio total: 190.0


## Problemas de PE con variables con doble subíndice

En problemas de optimización como los **problemas de asignación** o **problemas de transporte**, las variables de decisión suelen pueden doble subíndice para representar relaciones entre dos conjuntos (por ejemplo, orígenes y destinos).

A continuación, vamos a ver cómo modelar estas relaciones en Pyomo utilizando conjuntos, parámetros, y variables de decisión con doble subíndice.

### 1. Definir conjuntos y parámetros

Primero, definimos los **conjuntos** y los **parámetros** que representan las dimensiones de las matrices de datos (como costos, oferta, y demanda).

```python
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Conjuntos
model.I = Set(initialize=[1, 2, 3])  # Conjunto de orígenes
model.J = Set(initialize=[1, 2, 3])  # Conjunto de destinos

# Parámetros (ejemplo: matriz de costos)
cost = {(1, 1): 4, (1, 2): 8, (1, 3): 6,
        (2, 1): 5, (2, 2): 7, (2, 3): 4,
        (3, 1): 3, (3, 2): 9, (3, 3): 8}
model.cost = Param(model.I, model.J, initialize=cost)

# Parámetro de oferta y demanda
model.supply = Param(model.I, initialize={1: 20, 2: 30, 3: 50})
model.demand = Param(model.J, initialize={1: 30, 2: 40, 3: 30})
```

### 2. Definir las variables de decisión

Las **variables de decisión** con doble subíndice se definen utilizando un conjunto bidimensional. En el caso del problema de transporte, estas variables representan la cantidad enviada desde un origen $i$ a un destino $j$.

```python
# Variable de decisión (cantidad enviada de origen i a destino j)
model.x = Var(model.I, model.J, within=NonNegativeReals)
```

### 3. Definir la función objetivo

La **función objetivo** en problemas como el de transporte suele ser minimizar el costo total, que se calcula como la suma del producto entre los costos unitarios y las cantidades enviadas.

```python
# Función objetivo: minimizar el costo total
def objective_rule(model):
    return sum(model.cost[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = Objective(rule=objective_rule, sense=minimize)
```



### 4. Definir las restricciones

En el problema de transporte, las restricciones aseguran que:

1. La oferta desde cada origen no puede exceder su capacidad.
2. La demanda en cada destino debe ser completamente satisfecha.

#### Restricción 1: Oferta
$$\sum_{j \in J} x_{i,j} \leq 	a_{i}, \forall i \in I
$$

```python
# Restricciones de oferta
def supply_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.supply[i]
model.supply_constraint = Constraint(model.I, rule=supply_rule)
```

#### Restricción 2: Demanda
$$\sum_{i \in I} x_{i,j} \geq 	b_{j}, \quad \forall j \in J
$$

```python
# Restricciones de demanda
def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) >= model.demand[j]
model.demand_constraint = Constraint(model.J, rule=demand_rule)
```

In [2]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Conjuntos
model.I = Set(initialize=[1, 2, 3])  # Orígenes
model.J = Set(initialize=[1, 2, 3])  # Destinos

# Parámetros
cost = {(1, 1): 4, (1, 2): 8, (1, 3): 6,
        (2, 1): 5, (2, 2): 7, (2, 3): 4,
        (3, 1): 3, (3, 2): 9, (3, 3): 8}
model.cost = Param(model.I, model.J, initialize=cost)

model.supply = Param(model.I, initialize={1: 20, 2: 30, 3: 50})
model.demand = Param(model.J, initialize={1: 30, 2: 40, 3: 30})

# Variables de decisión
model.x = Var(model.I, model.J, within=NonNegativeReals)

# Función objetivo
def objective_rule(model):
    return sum(model.cost[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = Objective(rule=objective_rule, sense=minimize)

# Restricciones de oferta
def supply_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.supply[i]
model.supply_constraint = Constraint(model.I, rule=supply_rule)

# Restricciones de demanda
def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) >= model.demand[j]
model.demand_constraint = Constraint(model.J, rule=demand_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

# Imprimir los resultados
print("Resultados:")
for i in model.I:
    for j in model.J:
        if model.x[i, j]() > 0:
            print(f"Enviar {model.x[i, j]()} unidades de {i} a {j}")
print(f"Costo total: {model.obj()}")

Resultados:
Enviar 20.0 unidades de 1 a 2
Enviar 30.0 unidades de 2 a 3
Enviar 30.0 unidades de 3 a 1
Enviar 20.0 unidades de 3 a 2
Costo total: 550.0



### Ejercicio de Programación Entera con Pyomo

Una comunidad autónoma tiene 6 ciudades. Se pide que ayudes al presidente de la comunidad a decidir en qué ciudades situar parques de bomberos. El presidente desea ubicar el mínimo número de parques necesario para asegurar que siempre existe al menos un parque a como máximo 15 minutos de tiempo de desplazamiento de cada ciudad. Los tiempos en minutos necesarios para desplazarse entre ciudades se indican en la tabla siguiente:

|          | Ciudad 1 | Ciudad 2 | Ciudad 3 | Ciudad 4 | Ciudad 5 | Ciudad 6 |
|----------|----------|----------|----------|----------|----------|----------|
| **Ciudad 1** |    0     |   10     |   20     |   30     |   35     |   20     |
| **Ciudad 2** |   10     |    0     |   25     |   35     |   30     |   10     |
| **Ciudad 3** |   20     |   25     |    0     |   35     |   25     |   15     |
| **Ciudad 4** |   30     |   35     |   35     |    0     |   15     |   25     |
| **Ciudad 5** |   35     |   30     |   25     |   15     |    0     |   14     |
| **Ciudad 6** |   20     |   10     |   15     |   25     |   14     |    0     |

#### Apartado (a) 
Formula en detalle un problema de optimización que determine cuántos parques de bomberos debes construir y en qué ciudades deben ubicarse. Define las variables de decisión y formula la función objetivo y las restricciones.

**Pista**: Para cada ciudad debes añadir una restricción. Esta restricción, para cada ciudad, tiene que identificar las ciudades que se encuentran a 15 minutos o menos de tiempo de desplazamiento de dicha ciudad.

#### Apartado (b) 
Resuelva el problema de optimización utilizando Pyomo.

# APARTADO A)

### Formulación del Problema

Queremos determinar el número mínimo de parques de bomberos necesarios y en qué ciudades deben ubicarse, asegurando que cada ciudad esté cubierta por al menos un parque que se encuentre a **15 minutos o menos** de distancia.

---

#### Variables de Decisión
Definimos las variables binarias:

$$
y_j =
\begin{cases} 
1 & \text{si se construye un parque de bomberos en la ciudad } j, \\
0 & \text{en caso contrario.}
\end{cases}
$$

Donde \( j \in \{1, 2, 3, 4, 5, 6\} \).

---

#### Función Objetivo
Minimizar el número total de parques construidos:

$$
\text{Minimizar: } \sum_{j=1}^6 y_j
$$

---

#### Restricciones
Para garantizar que cada ciudad esté cubierta por al menos un parque de bomberos dentro de 15 minutos de desplazamiento, añadimos una restricción para cada ciudad \( i \):

$$
\sum_{j \in C(i)} y_j \geq 1, \quad \forall i \in \{1, 2, 3, 4, 5, 6\}
$$

Donde \( C(i) \) representa el conjunto de ciudades \( j \) que se encuentran a **15 minutos o menos** de la ciudad \( i \). Según la matriz de distancias proporcionada, los conjuntos \( C(i) \) son:

- \( C(1) = \{1, 2, 6\} \)
- \( C(2) = \{1, 2, 6\} \)
- \( C(3) = \{3, 6\} \)
- \( C(4) = \{4, 5\} \)
- \( C(5) = \{4, 5, 6\} \)
- \( C(6) = \{1, 2, 3, 5, 6\} \)

Por lo tanto, las restricciones son:

$$
y_1 + y_2 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 1)}
$$
$$
y_1 + y_2 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 2)}
$$
$$
y_3 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 3)}
$$
$$
y_4 + y_5 \geq 1 \quad \text{(Cobertura de la ciudad 4)}
$$
$$
y_4 + y_5 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 5)}
$$
$$
y_1 + y_2 + y_3 + y_5 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 6)}
$$

---

#### Restricciones de Binariedad
$$
y_j \in \{0, 1\}, \quad \forall j \in \{1, 2, 3, 4, 5, 6\}
$$

---

### Formulación Completa del Problema

**Minimizar:**
$$
\sum_{j=1}^6 y_j
$$

**Sujeto a:**

$$
y_1 + y_2 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 1)}
$$
$$
y_1 + y_2 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 2)}
$$
$$
y_3 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 3)}
$$
$$
y_4 + y_5 \geq 1 \quad \text{(Cobertura de la ciudad 4)}
$$
$$
y_4 + y_5 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 5)}
$$
$$
y_1 + y_2 + y_3 + y_5 + y_6 \geq 1 \quad \text{(Cobertura de la ciudad 6)}
$$

$$
y_j \in \{0, 1\}, \quad \forall j \in \{1, 2, 3, 4, 5, 6\}
$$


In [4]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Conjuntos
ciudades = [1, 2, 3, 4, 5, 6]

# Parámetros: tiempos de desplazamiento (en minutos)
distancias = {
    (1, 1): 0, (1, 2): 10, (1, 3): 20, (1, 4): 30, (1, 5): 35, (1, 6): 20,
    (2, 1): 10, (2, 2): 0, (2, 3): 25, (2, 4): 35, (2, 5): 30, (2, 6): 10,
    (3, 1): 20, (3, 2): 25, (3, 3): 0, (3, 4): 35, (3, 5): 30, (3, 6): 15,
    (4, 1): 30, (4, 2): 35, (4, 3): 35, (4, 4): 0, (4, 5): 25, (4, 6): 25,
    (5, 1): 35, (5, 2): 30, (5, 3): 30, (5, 4): 25, (5, 5): 0, (5, 6): 14,
    (6, 1): 20, (6, 2): 10, (6, 3): 15, (6, 4): 25, (6, 5): 14, (6, 6): 0
}

# Variables de decisión: si se coloca un parque en cada ciudad (binaria)
model.y = Var(ciudades, within=Binary)

# Restricciones: asegurar que cada ciudad tenga un parque a menos de 15 minutos
def cobertura_rule(model, i):
    return sum(model.y[j] for j in ciudades if distancias[i, j] <= 15) >= 1
model.cobertura = Constraint(ciudades, rule=cobertura_rule)

# Función objetivo: minimizar el número de parques
model.obj = Objective(expr=sum(model.y[j] for j in ciudades), sense=minimize)

# Resolver el modelo usando GLPK
solver = SolverFactory('glpk')
result = solver.solve(model)

# Imprimir los resultados
print("Ubicación de los parques de bomberos:")
for j in ciudades:
    if model.y[j]() > 0.5:  # La variable es binaria (0 o 1)
        print(f"Colocar un parque en la ciudad {j}")

print(f"Número mínimo de parques necesarios: {model.obj()}")


Ubicación de los parques de bomberos:
Colocar un parque en la ciudad 1
Colocar un parque en la ciudad 4
Colocar un parque en la ciudad 6
Número mínimo de parques necesarios: 3.0
